In [2]:
%cd ..

/home/abaric/phd/retriever-diachronic-emb


In [3]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from gensim.models import Word2Vec

def intersection_align_gensim(m1, m2):
    common = list(set(m1.wv.key_to_index).intersection(set(m2.wv.key_to_index)))
    common.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)

    def align_model(model):
        vectors = np.array([model.wv[w] for w in common])
        model.wv.vectors = vectors
        model.wv.key_to_index = {w: i for i, w in enumerate(common)}
        model.wv.index_to_key = common
        return model

    return align_model(m1), align_model(m2)

def smart_procrustes_align_gensim(base, other):
    base, other = intersection_align_gensim(base, other)
    m = other.wv.vectors.T @ base.wv.vectors
    u, _, v = np.linalg.svd(m)
    ortho = u @ v
    other.wv.vectors = other.wv.vectors @ ortho
    return other

def cosine_distance(a, b):
    return (1 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))) / 2

In [ ]:
# Load the model (replace X with 1, 2, or 3 as needed)
wave_1 = Word2Vec.load('../5ysplits_models/diachronic_wave_1_processed_model.bin')
wave_2 = Word2Vec.load('../5ysplits_models/diachronic_wave_2_processed_model.bin')
wave_3 = Word2Vec.load('../5ysplits_models/diachronic_wave_3_processed_model.bin')
wave_4 = Word2Vec.load('../5ysplits_models/diachronic_wave_4_processed_model.bin')
wave_5 = Word2Vec.load('../5ysplits_models/diachronic_wave_5_processed_model.bin')

In [ ]:
import json

word_counts_wave_1 = json.load(
    open(
        "../5ysplits_models/diachronic_wave_1_processed_word_counts.json"
    )
)
word_counts_wave_2 = json.load(
    open(
        "../5ysplits_models/diachronic_wave_2_processed_word_counts.json"
    )
)
word_counts_wave_3 = json.load(
    open(
        "../5ysplits_models/diachronic_wave_3_processed_word_counts.json"
    )
)
word_counts_wave_4 = json.load(
    open(
        "../5ysplits_models/diachronic_wave_4_processed_word_counts.json"
    )
)
word_counts_wave_5 = json.load(
    open(
        "../5ysplits_models/diachronic_wave_5_processed_word_counts.json"
    )
)

word_counts_separate = [
    word_counts_wave_1,
    word_counts_wave_2,
    word_counts_wave_3,
    word_counts_wave_4,
    word_counts_wave_5,
]

In [ ]:
waves = {0: wave_1, 1: wave_2, 2: wave_3, 3: wave_4, 4: wave_5}

In [ ]:
models = [waves[key] for key in sorted(waves.keys())]

periods = sorted(waves.keys())

for i in reversed(range(1, len(models))):
    models[i - 1] = smart_procrustes_align_gensim(models[i], models[i - 1])